# <font color="red">Hackday - Dados Inglórios</font>

## Planejamento da Solução

### Input

**Problema de Negócio**

A inclusão financeira continua a ser um dos principais obstáculos ao desenvolvimento econômico e humano na África. Por exemplo, no Quênia, Ruanda, Tanzânia e Uganda, apenas 9,1 milhões de adultos (14% dos adultos), têm acesso ou usam uma conta bancária comercial.

Em 2008, o nível de inclusão financeira na África Subsaariana era de pouco mais de 23%. Em 2018, esse número quase dobrou. No Togo, Mazamesso Assih, ministro de finanças, coordena uma estratégia de inclusão financeira com bancos parceiros. Ele afirma que garantir acesso a serviços financeiros básicos para a população, é fundamental para impulsionar a economia Africana.

Em 2022, conforme destacado no último relatório do Banco Central da África Ocidental, atingiu-se uma das mais altas taxas de inclusão financeira da região, próxima de 82%. Uma parcela significativa desse aumento veio do aumento dos serviços financeiros digitais.

Assih afirma também, que existem três razões principais pelas quais as nações africanas devem apostar na inclusão financeira:

 1. Tornar os serviços financeiros mais acessíveis promove o empoderamento das pessoas mais vulneráveis, especialmente as mulheres.
 2. Combater redes criminosas, passar de uma economia exclusivamente monetária para uma infraestrutura financeira digital torna mais fácil para as autoridades rastrear transações e lidar com contrabandistas e traficantes.
 3. impulsionará a conectividade dos setores público e privado para um crescimento sustentável, apoiando as start-ups africanas.
 
O objetivo desta competição é criar um modelo de aprendizado de máquina para prever quais indivíduos vão ou não utilizar uma conta bancária. Os modelos e soluções desenvolvidos devem fornecer uma indicação do estado da inclusão financeira no Quênia, Ruanda, Tanzânia e Uganda.



**Dados**

Os dados necessários podem ser encontrados no [repositório da atividade](https://www.kaggle.com/c/inclusao-financeira-na-africa). No arquivo train.csv você encontrará:

|Feature|Descrição|
|:------|:--------|
|country| Pais de Origem.
|year| Ano da Inclusão
|uniqueid| Id do Usuário.
|bank_account| Possui ou Não a Conta no Banco.
|location_type| Tipo de Localidade (Rural...)
|cellphone_access| Possui ou Não Celular.
|household_size| Tamanho da Casa.
|age_of_respondent| Idade do Usuário.
|gender_of_respondent| Gênero do Usuário.
|relationship_with_head| Status com o Dono da Conta.
|marital_status| Situação Civil.
|education_level| Nível de Educação.
|job_type| Tipo de Trabalho.

### Output

Um arquivo CSV com a previsão do modelo dizendo quais clientes abrirão uma conta digital em duas colunas:  

|uid|	bank_account|
|:--|:--------------|
|Rwanda_uniqueid_625|	No|
|Rwanda_uniqueid_1561|	No|
|Rwanda_uniqueid_4806|	No|
|Tanzania_uniqueid_4902|	Yes|
|Rwanda_uniqueid_980|	No|
|Rwanda_uniqueid_7043|	No|

### Tasks

- Descrição dos Dados
    - Renomear colunas
    - Checar dimenões
    - Checar tipos
    - Alterar tipos
    - Checar vazios
    - Preencher os vazios
    - Descrições Estatísticas

- Feature Engineering
    - Mapa de hipóteses
    - Criação de hipóteses
- Seleção de features (Filtro)
- EDA
    - Univariada
    - Bivariada (teste de hipóteses)
    - Multivariadas
- Preparação dos dados
    - Rescala das features
- Seleção de features
- Construção do modelo
    - modelos de classificação
    - guideline
- Avaliação do modelo
- Fine Tuning
- CSV Final

## 0. Settings

### 0.1. Imports

In [46]:
# manipulação de dados
import pandas as pd
import numpy as np

# visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import HTML

# scikit-learn
from sklearn import model_selection as ms
from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing as pp
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

### 0.2. Helper Functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display(HTML('<style>.container {width:100% !important;}</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    sns.set()

jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


### 0.3. Data Loading

In [3]:
train_file = './data/train.csv'

df_raw = pd.read_csv(train_file)
df_raw.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,uid,bank_account
0,Rwanda,2016,uniqueid_4858,Rural,Yes,6,45,Male,Head of Household,Divorced/Seperated,Primary education,Farming and Fishing,Rwanda_uniqueid_4858,No
1,Tanzania,2017,uniqueid_3015,Urban,No,4,33,Female,Head of Household,Single/Never Married,Primary education,Self employed,Tanzania_uniqueid_3015,No
2,Rwanda,2016,uniqueid_103,Rural,Yes,7,43,Male,Head of Household,Married/Living together,Secondary education,Farming and Fishing,Rwanda_uniqueid_103,No
3,Rwanda,2016,uniqueid_4582,Rural,No,6,35,Female,Head of Household,Married/Living together,Primary education,Farming and Fishing,Rwanda_uniqueid_4582,No
4,Tanzania,2017,uniqueid_2854,Urban,Yes,2,30,Male,Head of Household,Single/Never Married,Primary education,Informally employed,Tanzania_uniqueid_2854,No


## 1. Descrição dos Dados

In [4]:
df1 = df_raw.copy()

In [5]:
df1['country'].unique()

array(['Rwanda', 'Tanzania', 'Kenya', 'Uganda'], dtype=object)

In [6]:
df1['location_type'].unique()

array(['Rural', 'Urban'], dtype=object)

In [7]:
df1['relationship_with_head'].unique()

array(['Head of Household', 'Spouse', 'Child', 'Parent', 'Other relative',
       'Other non-relatives'], dtype=object)

In [8]:
df1['marital_status'].unique()

array(['Divorced/Seperated', 'Single/Never Married',
       'Married/Living together', 'Widowed', 'Dont know'], dtype=object)

In [9]:
df1['education_level'].unique()

array(['Primary education', 'Secondary education',
       'Vocational/Specialised training', 'Tertiary education',
       'No formal education', 'Other/Dont know/RTA'], dtype=object)

### 1.1. Renomear features

### 1.2. Checar Dimensões

### 1.3. Checar Tipos

### 1.4. Alterar Tipos

### 1.5. Checa NaN

In [10]:
df1.isna().sum()

country                   0
year                      0
uniqueid                  0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
uid                       0
bank_account              0
dtype: int64

### 1.6. Preenche NaN

### 1.7. Estatística Descritiva

## 2. Feature Engineering

In [11]:
df2 = df1.copy()

In [12]:
df2.dtypes

country                   object
year                       int64
uniqueid                  object
location_type             object
cellphone_access          object
household_size             int64
age_of_respondent          int64
gender_of_respondent      object
relationship_with_head    object
marital_status            object
education_level           object
job_type                  object
uid                       object
bank_account              object
dtype: object

In [13]:
count = df2['country'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['country'] = df2['country'].map(mapper)

In [14]:
count = df2['location_type'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['location_type'] = df2['location_type'].map(mapper)

In [15]:
count = df2['cellphone_access'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['cellphone_access'] = df2['cellphone_access'].map(mapper)

In [16]:
count = df2['gender_of_respondent'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['gender_of_respondent'] = df2['gender_of_respondent'].map(mapper)

In [17]:
count = df2['relationship_with_head'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['relationship_with_head'] = df2['relationship_with_head'].map(mapper)

In [18]:
count = df2['marital_status'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['marital_status'] = df2['marital_status'].map(mapper)

In [19]:
count = df2['education_level'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['education_level'] = df2['education_level'].map(mapper)

In [20]:
count = df2['job_type'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['job_type'] = df2['job_type'].map(mapper)

In [21]:
df2['bank_account'] = df2['bank_account'].apply(lambda x: 1 if x == 'Yes' else 0)

### 2.1. Mapa de Hipóteses

### 2.2. Criação de Hipóteses

## 3. Seleção de Features

In [23]:
df3 = df2.copy()

## 4. EDA

In [24]:
df4 = df3.copy()

### 4.1. Univariada

### 4.2. Bivariadas

### 4.3. Multivariadas

## 5. Preparação dos Dados

In [34]:
df5 = df4.copy()

In [35]:
df5.columns

Index(['country', 'year', 'uniqueid', 'location_type', 'cellphone_access',
       'household_size', 'age_of_respondent', 'gender_of_respondent',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'uid', 'bank_account'],
      dtype='object')

In [37]:
df5['country'] = pp.MinMaxScaler().fit_transform(df5[['country']].values)
df5['year'] = pp.MinMaxScaler().fit_transform(df5[['year']].values)
df5['location_type'] = pp.MinMaxScaler().fit_transform(df5[['location_type']].values)
df5['cellphone_access'] = pp.MinMaxScaler().fit_transform(df5[['cellphone_access']].values)
df5['household_size'] = pp.MinMaxScaler().fit_transform(df5[['household_size']].values)
df5['age_of_respondent'] = pp.MinMaxScaler().fit_transform(df5[['age_of_respondent']].values)
df5['gender_of_respondent'] = pp.MinMaxScaler().fit_transform(df5[['gender_of_respondent']].values)
df5['relationship_with_head'] = pp.MinMaxScaler().fit_transform(df5[['relationship_with_head']].values)
df5['marital_status'] = pp.MinMaxScaler().fit_transform(df5[['marital_status']].values)
df5['education_level'] = pp.MinMaxScaler().fit_transform(df5[['education_level']].values)
df5['job_type'] = pp.MinMaxScaler().fit_transform(df5[['job_type']].values)

### 5.1. Reescala dos Dados

## 6. Seleção de Features

In [38]:
df6 = df5.copy()

### 6.1. Separação de Treino e Teste

In [39]:
df6.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,uid,bank_account
0,1.000000,0.0,uniqueid_4858,1.0,1.0,0.25,0.345238,0.0,1.0,0.189722,1.00000,0.849111,Rwanda_uniqueid_4858,0
1,0.677648,0.5,uniqueid_3015,0.0,0.0,0.15,0.202381,1.0,1.0,0.730521,1.00000,1.000000,Tanzania_uniqueid_3015,0
2,1.000000,0.0,uniqueid_103,1.0,1.0,0.30,0.321429,0.0,1.0,1.000000,0.33015,0.849111,Rwanda_uniqueid_103,0
3,1.000000,0.0,uniqueid_4582,1.0,0.0,0.25,0.226190,1.0,1.0,1.000000,1.00000,0.849111,Rwanda_uniqueid_4582,0
4,0.677648,0.5,uniqueid_2854,0.0,1.0,0.05,0.166667,0.0,1.0,0.730521,1.00000,0.866900,Tanzania_uniqueid_2854,0


In [41]:
X = df6.drop(['uniqueid','uid','bank_account'],axis=1)
y = df6['bank_account']

x_train, x_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.3, random_state=42)

## 7. Construção do Modelo

In [52]:
k_fold=10
score_list = dict()

### 7.1. Random Forest Classifier

In [45]:
# criar o modelo
rfc_model = ensemble.RandomForestClassifier(random_state=42)

# treinar o modelo
rfc_model.fit(x_train, y_train)

# fazer a predição
yhat_rfc = rfc_model.predict(x_test)

# calcula a performance
metrics.f1_score(y_test, yhat_rfc)

0.4309927360774819

In [53]:
score = ms.cross_val_score(rfc_model,X,y,scoring='f1',cv=k_fold)
score_list['Random Forest']=np.mean(score)

## 7.2 Logistic Regression

In [42]:
# criar o modelo
logistic_model = linear_model.LogisticRegression()

# treinar o modelo
logistic_model.fit(x_train, y_train)

# fazer a predição
yhat_logistic = logistic_model.predict(x_test)

# calcula a performance
metrics.f1_score(y_test, yhat_logistic)


0.30387596899224806

In [54]:
score = ms.cross_val_score(logistic_model,X,y,scoring='f1',cv=k_fold)
score_list['Logistic Regression']=np.mean(score)

## 7.3 Support Vector Machine

In [44]:
# criar o modelo
svm_model = SVC()

# treinar o modelo
svm_model.fit(x_train, y_train)

# fazer a predição
yhat_svm = svm_model.predict(x_test)

# calcula a performance
metrics.f1_score(y_test, yhat_svm)

0.2916666666666667

In [55]:
score = ms.cross_val_score(svm_model,X,y,scoring='f1',cv=k_fold)
score_list['SVM']=np.mean(score)

## 7.4 KNN

In [47]:
# criar o modelo
knn_model = KNeighborsClassifier()

# treinar o modelo
knn_model.fit(x_train, y_train)

# fazer a predição
yhat_knn = knn_model.predict(x_test)

# calcula a performance
metrics.f1_score(y_test, yhat_knn)

0.4168797953964194

In [56]:
score = ms.cross_val_score(knn_model,X,y,scoring='f1',cv=k_fold)
score_list['KNN']=np.mean(score)

## 8. Avaliação do Modelo

In [57]:
score_list

{'Random Forest': 0.43730310917946413,
 'Logistic Regression': 0.3169443271918454,
 'SVM': 0.3264722477786753,
 'KNN': 0.42815759142999543}

## 9. Fine Tunning

## 10. Output da Submissão

In [60]:
x_test.head()

,year,household_size,age_of_respondent
3717,2016,6,48
11036,2016,3,27
2088,2017,4,32
1399,2018,3,19
1178,2017,2,36


In [61]:
y_test[0:5]

3717     0
11036    0
2088     0
1399     0
1178     0
Name: bank_account, dtype: int64